In [69]:
import ee

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [70]:
!pip install branca

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
import numpy as np
import folium
from folium import plugins
import branca
from branca.element import Figure
import datetime

In [72]:
#@title ee Mapper folium function
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name, opacity=1):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            opacity = opacity,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            opacity = opacity,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            opacity = opacity,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            opacity = opacity,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [73]:
batas_Indonesia_FC = ee.FeatureCollection("USDOS/LSIB/2017")\
.filter(ee.Filter.eq('COUNTRY_NA', 'Indonesia')).geometry()

batas_provinsi = ee.FeatureCollection("FAO/GAUL/2015/level1")\
.filter(ee.Filter.eq('ADM1_NAME','Papua')).geometry()

batas_wilayah = batas_provinsi

#Koleksi_Sentinel = ee.ImageCollection("MODIS/006/MCD19A2_GRANULES")\
#.filterBounds(batas_wilayah)

#Koleksi_Sentinel = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_AER_AI")\
#.filterBounds(batas_wilayah)

#Koleksi_Sentinel = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_AER_AI")\
#.filterBounds(batas_wilayah)

Koleksi_Sentinel = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_SO2')\
.filterBounds(batas_wilayah)

#data_cuaca = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY")\
#.filterBounds(batas_wilayah)

data_cuaca = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")\
.filterBounds(batas_wilayah)

Tahun_terlampau = 2019

tanggal_hari_ini = datetime.date.today()
tanggal_ini = tanggal_hari_ini.day
bulan_ini = tanggal_hari_ini.month
tahun_ini = tanggal_hari_ini.year
#kanal_Aerosol = 'Optical_Depth_047'
kanal_Aerosol = 'absorbing_aerosol_index'
#kanal_suhu = 'temperature_2m'
kanal_suhu = 'Tair_f_inst'

In [74]:
def bersihkanQA(citra):
  masker_QA = citra.select('AOD_QA').eq(1)\
  .Or(citra.select('AOD_QA').eq(865)\
      .Or(citra.select('AOD_QA').eq(1057)\
          .Or(citra.select('AOD_QA').eq(2)\
              .Or(citra.select('AOD_QA').eq(4002)))))
  return citra.mask(masker_QA)

def kurangi_resolusi(citra):
  citra2 = citra.reduceResolution(**{
      'reducer': ee.Reducer.mean(),
      'maxPixels': 1024
    })
  return citra2

In [75]:
tanggal_Awal = '2021-01-01'
tanggal_Akhir = '2022-01-01'
## tanggal_Akhir = ee.Date.fromYMD(tahun_ini, bulan_ini-1, 31)

#Aerosol_modus = ee.Image(Koleksi_Sentinel\
#                         .filter(ee.Filter.date(tanggal_Awal, tanggal_Akhir))\
#                         .map(bersihkanQA)\
#                         .map(kurangi_resolusi)\
#                         .select(kanal_Aerosol)\
#                         .mode()).multiply(0.001)

Aerosol_modus = ee.Image(Koleksi_Sentinel\
                         .filter(ee.Filter.date(tanggal_Awal, tanggal_Akhir))\
                         .map(kurangi_resolusi)\
                         .select(kanal_Aerosol)\
                         .mean())

Suhu_max_bulanan = ee.Image(data_cuaca\
                         .filter(ee.Filter.date(tanggal_Awal, tanggal_Akhir))\
                         .select(kanal_suhu)\
                         .max()).subtract(273.15)

Suhu_min_bulanan = ee.Image(data_cuaca\
                         .filter(ee.Filter.date(tanggal_Awal, tanggal_Akhir))\
                         .select(kanal_suhu)\
                         .min()).subtract(273.15)

proy_suhu_max_bln = Suhu_max_bulanan.projection()
proy_suhu_min_bln = Suhu_min_bulanan.projection()

Suhu_Max = Suhu_max_bulanan.select(kanal_suhu).reduceRegion(**{
    'reducer':ee.Reducer.max(),
    'geometry':batas_Indonesia_FC,
    'scale':10000,
    'crs':proy_suhu_max_bln.crs(),
    #'crsTransform':proy_suhu_max_bln.getInfo()["transform"]
})

Suhu_Min = Suhu_min_bulanan.select(kanal_suhu).reduceRegion(**{
    'reducer':ee.Reducer.min(),
    'geometry':batas_Indonesia_FC,
    'scale':10000,
    'crs':proy_suhu_max_bln.crs(),
    #'crsTransform':proy_suhu_max_bln.getInfo()["transform"]
})

Suhu_Max = Suhu_Max.getInfo()[kanal_suhu]
Suhu_Max = np.asarray(Suhu_Max)

print(type(Suhu_Max))

Suhu_Min = Suhu_Min.getInfo()[kanal_suhu]
Suhu_Min = np.array(Suhu_Min)

area_cozy = Suhu_max_bulanan\
.select(kanal_suhu).lte(25)\
.And(Suhu_min_bulanan.select(kanal_suhu).gte(15));

#Aerosol_modus = Aerosol_modus.mask(area_cozy)

langit_bersih = Aerosol_modus.lte(0)

#Aerosol_modus = Aerosol_modus.mask(langit_bersih)

<class 'numpy.ndarray'>


In [76]:
AOD_tBesar = Aerosol_modus.reduceRegion(**{
  'reducer':ee.Reducer.max(),
  'geometry': batas_wilayah,
  'scale': 10000,
  'bestEffort': True
})

AOD_tBesar = AOD_tBesar.get(kanal_Aerosol).getInfo();
print(AOD_tBesar);

-1.2657470632256604


In [77]:
AOD_tKecil = Aerosol_modus.reduceRegion(**{
  'reducer':ee.Reducer.min(),
  'geometry': batas_wilayah,
  'scale': 10000,
  'bestEffort': True
})

AOD_tKecil = AOD_tKecil.get(kanal_Aerosol).getInfo();
print(AOD_tKecil);

-3.983762982021553


In [78]:
skala_warna_aerosol = {
  'bands': [kanal_Aerosol],
  ##min: Math.ceil(AOD_tKecil*1000)/1000,
  ##max: 0.1,
  ##bands:['array'],
  'min': AOD_tKecil,
  'max': AOD_tBesar,
  ##palet lihat di coolors.co
  'palette': ["#0081ff","#a4effb","#ffffff","#ffe49e","#cda431"]
}

skala_warna_temp = {
  ##min: Math.ceil(AOD_tKecil*1000)/1000,
  ##max: 0.1,
  ##bands:['array'],
  'bands':[kanal_suhu],
  'min': Suhu_Min[()],
  'max': Suhu_Max[()],
  ##palet lihat di coolors.co
  'palette': ["#3517fc","#17bebb","#09fd09","#ffc914","#f33823"]
}

# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'OpenTopoMap': folium.TileLayer(
        tiles = 'https://c.tile.opentopomap.org/{z}/{x}/{y}.png',
        attr = 'OpenTopoMap',
        name = 'OSM General',
        overlay = True,
        control = True,
    ),
    'ESRI Shaded Relief': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Shaded_Relief/MapServer/tile/{z}/{y}/{x}',
        attr = 'OpenStreetMap',
        name = 'OSM General',
        overlay = True,
        control = True,
        opacity = 0.5
    ),
    'ESRI Light Grey': folium.TileLayer(
        tiles = 'http://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}',
        attr = 'OpenStreetMap',
        name = 'OSM General',
        overlay = True,
        control = True
    )
}

In [79]:
## Point
bujur = 137.6413
lintang = -3.9722
point = ee.Geometry.Point([bujur, lintang]);

kebeningan_at_point = Aerosol_modus.reduceRegion(**{
  'reducer': ee.Reducer.first(),
  'geometry': point,
  'scale': 1000
})

kebeningan_at_point = ee.Number(kebeningan_at_point).getInfo()
kebeningan_at_point = kebeningan_at_point[kanal_Aerosol]

center = batas_wilayah.centroid().getInfo()['coordinates']
center.reverse()

# Create a folium map object.
my_map = folium.Map(location=center, zoom_start=6, width=800, height=450)

# Add custom basemaps
basemaps['ESRI Light Grey'].add_to(my_map)
basemaps['ESRI Shaded Relief'].add_to(my_map)

# Add the LANDSAT data to the map object.
my_map.add_ee_layer(Aerosol_modus.clip(batas_wilayah), skala_warna_aerosol, 'Rata-rata Indeks Aerosol', opacity=0.5)

#specify the min and max values of your data
num_idx_cmap = len(skala_warna_aerosol['palette'])
index_colormap = np.linspace(AOD_tKecil, AOD_tBesar, num_idx_cmap).tolist()
colormap = branca.colormap.LinearColormap(skala_warna_aerosol['palette'],
                                          index=index_colormap,
                                          vmin=AOD_tKecil, vmax=AOD_tBesar)
colormap.caption = 'Kebeningan langit'
colormap.add_to(my_map)

teks_kebeningan = folium.Html("<b>ILAGA:</b> <br> kebeningan langit <br><b>" + '{:.1f}'.format(kebeningan_at_point) + "</b>",
                        script=True)
popup = folium.Popup(teks_kebeningan, max_width=120)

folium.Marker(
    location=[lintang, bujur],
    popup=popup,
    icon=folium.Icon(color="darkblue", icon="info-sign"),
).add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
# display(my_map)
fig = Figure(width=800, height=450)
fig.add_child(my_map)

In [80]:
temp_at_point = Suhu_max_bulanan.reduceRegion(**{
  'reducer': ee.Reducer.first(),
  'geometry': point,
  'scale': 1000
})

temp_at_point = ee.Number(temp_at_point).getInfo()
temp_at_point = temp_at_point[kanal_suhu]

print(temp_at_point)

center = batas_wilayah.centroid().getInfo()['coordinates']
center.reverse()

# Create a folium map object.
my_map = folium.Map(location=center, zoom_start=6, width=800, height=450)

# Add custom basemaps
# basemaps['OSM General'].add_to(my_map)

# Add the LANDSAT data to the map object.
my_map.add_ee_layer(Suhu_max_bulanan.clip(batas_Indonesia_FC), skala_warna_temp, 'Suhu Maksimum Tahunan', opacity=0.5)

#specify the min and max values of your data
num_idx_cmap = len(skala_warna_temp['palette'])
index_colormap = np.linspace(Suhu_Min, Suhu_Max, num_idx_cmap).tolist()
colormap = branca.colormap.LinearColormap(skala_warna_temp['palette'],
                                          index=index_colormap,
                                          vmin=Suhu_Min, vmax=Suhu_Max)
colormap.caption = 'Suhu Maksimum Tahunan'
colormap.add_to(my_map)

teks_suhu = folium.Html("<b>ILAGA: </b><br> suhu udara <br><b>" + '{:.1f}'.format(temp_at_point) + "°C </b>",
                        script=True)
popup = folium.Popup(teks_suhu, max_width=75)

folium.Marker(
    location=[lintang, bujur],
    popup=popup,
    icon=folium.Icon(color="orange", icon="info-sign"),
).add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
# display(my_map)
fig = Figure(width=800, height=450)
fig.add_child(my_map)

21.035577392578148


In [81]:
temp_at_point = Suhu_min_bulanan.reduceRegion(**{
  'reducer': ee.Reducer.first(),
  'geometry': point,
  'scale': 1000
})

temp_at_point = ee.Number(temp_at_point).getInfo()
temp_at_point = temp_at_point[kanal_suhu]

print(temp_at_point)

center = batas_wilayah.centroid().getInfo()['coordinates']
center.reverse()

# Create a folium map object.
my_map = folium.Map(location=center, zoom_start=6, width=800, height=450)

# Add custom basemaps
# basemaps['OSM General'].add_to(my_map)

# Add the LANDSAT data to the map object.
my_map.add_ee_layer(Suhu_min_bulanan.clip(batas_Indonesia_FC), skala_warna_temp, 'Suhu Minimum Tahunan', opacity=0.5)

#specify the min and max values of your data
num_idx_cmap = len(skala_warna_temp['palette'])
index_colormap = np.linspace(Suhu_Min, Suhu_Max, num_idx_cmap).tolist()
colormap = branca.colormap.LinearColormap(skala_warna_temp['palette'],
                                          index=index_colormap,
                                          vmin=Suhu_Min, vmax=Suhu_Max)
colormap.caption = 'Suhu Minimum Tahunan'
colormap.add_to(my_map)

teks_suhu = folium.Html("<b>ILAGA: </b><br> suhu udara <br><b>" + '{:.1f}'.format(temp_at_point) + "°C </b>",
                        script=True)
popup = folium.Popup(teks_suhu, max_width=75)

folium.Marker(
    location=[lintang, bujur],
    popup=popup,
    icon=folium.Icon(color="green", icon="info-sign"),
).add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
# display(my_map)
fig = Figure(width=800, height=450)
fig.add_child(my_map)

6.567651367187523
